In [1]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import parser
import csv
import os
import json
from urllib import request
import re
from tqdm import tqdm
import numpy as np

In [33]:
import pandas as pd
#initialization of urls
urls = pd.read_csv('skytrax.csv')
urls = list(urls['Jump-href'])
urls = [url + '/?sortby=post_date%3ADesc&pagesize=100' for url in urls]
urls = [url.replace('airline-','seat-') for url in urls]
urls

['https://www.airlinequality.com/seat-reviews/thai-lion-air/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/sky-express-airlines/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/air-botswana/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/south-african-airways/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/latam-airlines/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/fiji-airways/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/nature-air/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/tibet-airlines/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/nordwind-airlines/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/seat-reviews/lam-mozambique-airlines/?sortby=post_d

In [3]:
def get_one_page_reviews(url):
    '''
    params:one url of certain page
    output:a df containing three conlumns 
    '''
    req = requests.get(url)
    soup = BeautifulSoup(req.text,"lxml")
    tables = soup.select('table')
    type_traveller = []
    aircraft = []
    seat_type = []
    for table in tables:
        if not table == tables[0]: # discard the first table
            stats = pd.read_html(table.prettify())
            stats = stats[0].set_index(0)
            if 'Aircraft' in stats.index:
                aircraft.append(stats.loc['Aircraft',1])
            else:
                aircraft.append(np.nan)
            if 'Type Of Traveller' in stats.index:
                type_traveller.append(stats.loc['Type Of Traveller',1])
            else:
                type_traveller.append(np.nan)
            if 'Seat Type' in stats.index:
                seat_type.append(stats.loc['Seat Type',1])
            else:
                seat_type.append(np.nan) 
    
    text_header = []
    text_body = []
    text_seattype = []
    ratings = []
    reviews = pd.DataFrame()
    airline = url.split('/')[4]
    reviews['airline'] = 1
    
    for article in soup.find_all('article',attrs={'itemprop':"review"}):
        header_soup = article.find_all('h2')
        body_soup = article.find_all('div',attrs={'class':'text_content','itemprop':'reviewBody'})
        seat_soup = article.find_all('td',attrs={'class':"review-rating-header cabin_flown "})
        rating_soup = article.find_all('span',attrs={'itemprop':'ratingValue'})
        for header in header_soup:
            text_header.append(header.text)
        for body in body_soup:
            text_body.append(body.text)
        for seat in seat_soup:
            text_seat.append(seat.text)
        for rating in rating_soup:
            ratings.append(rating.text)
            
    reviews['header'] = text_header
    reviews['body'] = text_body
    reviews['airline'] = airline
    reviews['type_traveller'] = type_traveller
    reviews['aircraft'] = aircraft
    reviews['seat_type'] = seat_type
    reviews['rating'] = ratings
    
    return reviews

In [24]:
url = 'https://www.airlinequality.com/airline-reviews/air-botswana/?sortby=post_date%3ADesc&pagesize=100'
req = requests.get(url)
soup = BeautifulSoup(req.text,"lxml")
ratings = soup.find_all('div',attrs={'class':'rating-10'})
ratings
for rating in ratings[1:]:
    rating_score = rating.find_all('span',attrs={'itemprop':'ratingValue'})
    print(rating_score)

[<span itemprop="ratingValue">1</span>]
[<span itemprop="ratingValue">2</span>]
[<span itemprop="ratingValue">9</span>]
[<span itemprop="ratingValue">10</span>]
[]
[<span itemprop="ratingValue">10</span>]
[<span itemprop="ratingValue">8</span>]
[<span itemprop="ratingValue">10</span>]
[<span itemprop="ratingValue">8</span>]
[<span itemprop="ratingValue">8</span>]
[<span itemprop="ratingValue">4</span>]
[<span itemprop="ratingValue">4</span>]
[<span itemprop="ratingValue">4</span>]
[<span itemprop="ratingValue">7</span>]
[<span itemprop="ratingValue">3</span>]
[<span itemprop="ratingValue">1</span>]
[<span itemprop="ratingValue">4</span>]
[]
[]
[]
[]
[]


In [34]:
def get_one_page_reviews(url):
    '''
    params:one url of certain page
    output:a df containing 7 conlumns 
    '''
    req = requests.get(url)
    soup = BeautifulSoup(req.text,"lxml")
    tables = soup.select('table')
    type_traveller = []
    aircraft = []
    seat_type = []
    for table in tables:
        if not table == tables[0]: # discard the first table
            stats = pd.read_html(table.prettify())
            stats = stats[0].set_index(0)
            if 'Aircraft' in stats.index:
                aircraft.append(stats.loc['Aircraft',1])
            else:
                aircraft.append(np.nan)
            if 'Type Of Traveller' in stats.index:
                type_traveller.append(stats.loc['Type Of Traveller',1])
            else:
                type_traveller.append(np.nan)
            if 'Seat Type' in stats.index:
                seat_type.append(stats.loc['Seat Type',1])
            else:
                seat_type.append(np.nan) 
    
    text_header = []
    text_body = []
    reviews = pd.DataFrame()
    airline = url.split('/')[4]
    reviews['airline'] = 1  
    for article in soup.find_all('article',attrs={'itemprop':"review"}):
        header_soup = article.find_all('h2')
        body_soup = article.find_all('div',attrs={'class':'text_content','itemprop':'reviewBody'})
        for header in header_soup:
            text_header.append(header.text)
        for body in body_soup:
            text_body.append(body.text)

    reviews['header'] = text_header
    reviews['body'] = text_body
    reviews['airline'] = airline
    reviews['type_traveller'] = type_traveller
    reviews['aircraft'] = aircraft
    reviews['seat_type'] = seat_type
    return reviews

def get_reviews_full(urls):
    REVIEWS_list = []
    for url in tqdm(urls):
        airline = url.split('/')[4]
        reviews_list = []
        r = requests.get(url)
        page_soup = BeautifulSoup(r.text,"lxml")
        review_count = page_soup.find('span',attrs={'itemprop':'reviewCount'}).text
        review_num = int(re.sub(r'\D','',review_count))
        page_num = review_num//100 + 2
        page_num

        sub_urls = [url]
        if page_num > 2:
            for i in range(2,page_num):
                url1,url2 = url.split('?')
                cache = 'page/{}/?'.format(i)
                sub_url = url1+cache+url2
                sub_urls.append(sub_url)

        for sub_url in sub_urls:
            review_one_page = get_one_page_reviews(sub_url)
            reviews_list.append(review_one_page)

        reviews_sum = pd.concat(reviews_list,axis = 0)
        REVIEWS_list.append(reviews_sum)

    REVIEWS = pd.concat(REVIEWS_list,axis=0)
    return REVIEWS
REVIEWS = get_reviews_full(urls)


 25%|████████████████████▏                                                           | 131/519 [02:34<09:00,  1.39s/it]


 51%|████████████████████████████████████████▌                                       | 263/519 [05:37<05:44,  1.35s/it]


 76%|████████████████████████████████████████████████████████████▉                   | 395/519 [08:23<02:17,  1.11s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 519/519 [10:44<00:00,  1.24s/it]


In [35]:
REVIEWS

,airline,header,body,type_traveller,aircraft,seat_type
0,south-african-airways,"""Really poor legroom""",✅ Trip Verified | A330-300. Positives : Good ...,Couple Leisure,NaN,Economy Class
1,south-african-airways,"""seats very uncomfortable""",✅ Verified Review | Booked the bulkhead row 6...,Couple Leisure,NaN,Economy Class
2,south-african-airways,"""I don't need more leg space""",Checked in online so be sure we would sit toge...,Couple Leisure,NaN,Economy Class
3,south-african-airways,"""recipe for a very uncomfortable ride""",Avoid at all costs seats 37A/B and 38A/B. Row ...,NaN,NaN,Economy Class
4,south-african-airways,"""mediocre seat""",Mediocre seat. Flew from JFK to JNB. Very litt...,NaN,NaN,Economy Class
...,...,...,...,...,...,...
3,aer-lingus,"""considerable discomfort""",I recently flew home to Ireland on the 24th of...,Solo Leisure,NaN,Economy Class
4,aer-lingus,"""point A to point B""",We were in the seats about halfway back in Eco...,Couple Leisure,NaN,Economy Class
5,aer-lingus,"""don't recline very much""","The seats on the A330 were okay to the USA, th...",Solo Leisure,NaN,Economy Class
6,aer-lingus,"""seats are a bit tight but bearable""",The seats are a bit tight but bearable. If you...,Couple Leisure,NaN,Economy Class


In [36]:
#output reviews csv
REVIEWS.to_csv('reviews_seat.csv')

In [27]:
REVIEWS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107928 entries, 0 to 4
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   airline         107928 non-null  object
 1   header          107928 non-null  object
 2   body            107928 non-null  object
 3   type_traveller  69253 non-null   object
 4   aircraft        31325 non-null   object
 5   seat_type       104906 non-null  object
dtypes: object(6)
memory usage: 5.8+ MB


In [6]:
REVIEWS

,airline,header,body
0,south-african-airways,"""Really poor legroom""",✅ Trip Verified | A330-300. Positives : Good ...
1,south-african-airways,"""seats very uncomfortable""",✅ Verified Review | Booked the bulkhead row 6...
2,south-african-airways,"""I don't need more leg space""",Checked in online so be sure we would sit toge...
3,south-african-airways,"""recipe for a very uncomfortable ride""",Avoid at all costs seats 37A/B and 38A/B. Row ...
4,south-african-airways,"""mediocre seat""",Mediocre seat. Flew from JFK to JNB. Very litt...
...,...,...,...
3,aer-lingus,"""considerable discomfort""",I recently flew home to Ireland on the 24th of...
4,aer-lingus,"""point A to point B""",We were in the seats about halfway back in Eco...
5,aer-lingus,"""don't recline very much""","The seats on the A330 were okay to the USA, th..."
6,aer-lingus,"""seats are a bit tight but bearable""",The seats are a bit tight but bearable. If you...


In [ ]:
###for coockies

def getXMLText(url):
    headers = {
        'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
        'Cookie': '' #这里要自己用f12获取自己登陆的微博cookie
    }
    try:
        r = requests.get(url , headers = headers)
        r.raise_for_status()
        r.encoding = "utf-8"
        return r.text
    except:
        return ""
url_tripadvisor = 'https://www.airlinequality.com/airline-reviews/ab-aviation'
tripadvisor = getXMLText(url_tripadvisor)
